# Import libraries

In [815]:
import pandas as pd
import pyodbc
import numpy as np
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

# Get datas from database

In [816]:
server = 'DESKTOP-4I8KM5F'
database = 'cars_data'
driver = 'ODBC Driver 17 for SQL Server'
connection_string = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

conn = pyodbc.connect(connection_string)

In [817]:
def get_table(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE' AND table_name LIKE '%[_]%'")
    tables = cursor.fetchall()
    cursor.close()
    return [table[0] for table in tables]

In [818]:
car_tables = get_table(conn)
dfs = {}

In [819]:
for table in car_tables:
    query = f"SELECT * FROM [{table}]"
    try:
        df = pd.read_sql(query, conn)
        dfs[table] = df
        print(f"Data from table '{table}' retrieved successfully")
    
    except Exception as e:
        print(f"Error retrieving data from table '{table}': {e}")

conn.close()

Data from table 'bonbanh_inf' retrieved successfully
Data from table 'newcar_inf' retrieved successfully
Data from table 'used_Car' retrieved successfully
Data from table 'final_data' retrieved successfully


C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\1449589249.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [820]:
df_bonbanh_inf = dfs['bonbanh_inf']
df_used_Car = dfs['used_Car']
df_newcar_inf = dfs['newcar_inf']

# Pre-processing

## New cars

In [821]:
df_newcar_inf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              587 non-null    object
 1   link              587 non-null    object
 2   manufacturer      587 non-null    object
 3   origin            587 non-null    object
 4   body_type         587 non-null    object
 5   seating_capacity  587 non-null    object
 6   engine            587 non-null    object
 7   torque            587 non-null    object
 8   transmission      587 non-null    object
 9   max_power         587 non-null    object
 10  drive             587 non-null    object
 11  fuel_type         587 non-null    object
 12  fuel_capacity     587 non-null    object
dtypes: object(13)
memory usage: 59.7+ KB


### Replace None

In [822]:
df_newcar_inf.replace({'None': np.nan}, inplace=True)
df_newcar_inf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              587 non-null    object 
 1   link              587 non-null    object 
 2   manufacturer      586 non-null    object 
 3   origin            587 non-null    object 
 4   body_type         587 non-null    object 
 5   seating_capacity  587 non-null    object 
 6   engine            587 non-null    object 
 7   torque            0 non-null      float64
 8   transmission      587 non-null    object 
 9   max_power         530 non-null    object 
 10  drive             587 non-null    object 
 11  fuel_type         585 non-null    object 
 12  fuel_capacity     587 non-null    object 
dtypes: float64(1), object(12)
memory usage: 59.7+ KB


C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\2749820825.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_newcar_inf.replace({'None': np.nan}, inplace=True)


### Fuel_capacity

In [823]:
df_newcar_inf['fuel_capacity'].unique()

array(['35 Lit', '33 Lit', '43 Lit', '42 Lit', '32 Lit', '0 Lit',
       '45 Lit', '40 Lit', '41 Lit', '37 Lit', '48 Lit', '44 Lit',
       '52 Lit', '50 Lit', '58 Lit', '80 Lit', '51 Lit', '46 Lit',
       '79 Lit', '55 Lit', '65 Lit', '76 Lit', '53 Lit', '47 Lit',
       '75 Lit', '54 Lit', '70 Lit', '62 Lit', '72 Lit', '60 Lit',
       '56 Lit', '78 Lit', '68 Lit', '57 Lit', '71 Lit', '103 Lit',
       '26 Lit', '85 Lit', '73 Lit', '66 Lit', '59 Lit', '88 Lit',
       '87 Lit', '67 Lit', '83 Lit', '82 Lit', '93 Lit', '90 Lit',
       '143 Lit', '89 Lit', '64 Lit', '104 Lit', '98 Lit', '84 Lit',
       '99 Lit', '77 Lit', '96 Lit'], dtype=object)

In [824]:
filtered_cars = df_newcar_inf[df_newcar_inf['fuel_capacity'] == '0 Lit']

car_names = filtered_cars['name']
print(car_names)

21     Vinfast Fadil 1.4 Tieu chuan CVT
175                Toyota Venza 2.5 CVT
244                Honda CR-V 1.5E 2021
248            Hyundai SantaFe 2.4 Xang
263                Honda CR-V 1.5G 2021
273                Honda CR-V 1.5L 2021
302      Toyota Fortuner 2.4 TRD AT 4X2
552                  BMW i8 1.5L Hybrid
585            Rolls-Royce Cullinan V12
586                Rolls-Royce Dawn V12
Name: name, dtype: object


In [825]:
df_newcar_inf.loc[df_newcar_inf['name'] == 'Vinfast Fadil 1.4 Tieu chuan CVT', 'fuel_capacity'] = '32 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Toyota Venza 2.5 CVT',  'fuel_capacity'] = '55 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Honda CR-V 1.5E 2021',  'fuel_capacity'] = '57 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Hyundai SantaFe 2.4 Xang',  'fuel_capacity'] = '71 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Honda CR-V 1.5G 2021',  'fuel_capacity'] = '57 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Honda CR-V 1.5L 2021',  'fuel_capacity'] = '57 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Toyota Fortuner 2.4 TRD AT 4X2',  'fuel_capacity'] = '80 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'BMW i8 1.5L Hybrid',  'fuel_capacity'] = '42 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Rolls-Royce Cullinan V12',  'fuel_capacity'] = '83 Lit'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Rolls-Royce Dawn V12',  'fuel_capacity'] = '78 Lit'

In [826]:
filtered_cars = df_newcar_inf[df_newcar_inf['fuel_capacity'] == '0 Lit']

car_names = filtered_cars['name']
print(car_names)

Series([], Name: name, dtype: object)


In [827]:
df_newcar_inf['fuel_capacity'] = df_newcar_inf['fuel_capacity'].str.replace(' Lit', '', regex=False).astype(int)
df_newcar_inf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              587 non-null    object 
 1   link              587 non-null    object 
 2   manufacturer      586 non-null    object 
 3   origin            587 non-null    object 
 4   body_type         587 non-null    object 
 5   seating_capacity  587 non-null    object 
 6   engine            587 non-null    object 
 7   torque            0 non-null      float64
 8   transmission      587 non-null    object 
 9   max_power         530 non-null    object 
 10  drive             587 non-null    object 
 11  fuel_type         585 non-null    object 
 12  fuel_capacity     587 non-null    int64  
dtypes: float64(1), int64(1), object(11)
memory usage: 59.7+ KB


### fuel_type

In [828]:
df_newcar_inf['fuel_type'].unique()

array(['Xang', 'Dau', nan, 'Dien'], dtype=object)

In [829]:
filtered_cars = df_newcar_inf[df_newcar_inf['fuel_type'].isna()]

car_names = filtered_cars['name']
print(car_names)

178        Honda Civic 1.8 G
245    Honda CR-V 1.5 E 2019
Name: name, dtype: object


In [830]:
df_newcar_inf.loc[df_newcar_inf['name'] == 'Honda Civic 1.8 G', 'fuel_type'] = 'Xang'
df_newcar_inf.loc[df_newcar_inf['name'] == 'Honda CR-V 1.5 E 2019',  'fuel_type'] = 'Xang'
df_newcar_inf['fuel_type'].unique()

array(['Xang', 'Dau', 'Dien'], dtype=object)

### max_power, torque, link

In [831]:
df_newcar_inf.drop(columns=['max_power', 'torque', 'link'], inplace=True)

### manufacturer

In [832]:
df_newcar_inf['manufacturer'].unique()

array(['Kia', 'Suzuki', 'Toyota', 'Hyundai', 'Chevrolet', 'Mitsubishi',
       'VinFast', 'Honda', 'Nissan', 'Mazda', 'Ford', 'Renault', 'Isuzu',
       'Volkswagen', 'Subaru', 'Peugeot', 'Land Rover', 'Mercedes',
       'Audi', 'BMW', 'Mini', 'Volvo', 'Jaguar', 'Lexus', 'Infiniti',
       'Porsche', nan, 'Cadillac', 'Lincoln', 'Maserati',
       'Mercedes Maybach', 'McLaren', 'Aston Martin', 'Bentley',
       'Rolls Royce'], dtype=object)

In [833]:
filtered_cars = df_newcar_inf[df_newcar_inf['manufacturer'].isna()]
filtered_cars['name']

464    Land Rover Range Rover Evoque 2.0L I4 Turbocha...
Name: name, dtype: object

In [834]:
df_newcar_inf.loc[464, 'manufacturer'] = 'Land Rover'

In [835]:
df_newcar_inf['manufacturer'] = df_newcar_inf['manufacturer'].replace('Mercedes Maybach', 'Mercedes')
df_newcar_inf['manufacturer'] = df_newcar_inf['manufacturer'].replace('Rolls Royce', 'Rolls-Royce')
df_newcar_inf['manufacturer'].unique()

array(['Kia', 'Suzuki', 'Toyota', 'Hyundai', 'Chevrolet', 'Mitsubishi',
       'VinFast', 'Honda', 'Nissan', 'Mazda', 'Ford', 'Renault', 'Isuzu',
       'Volkswagen', 'Subaru', 'Peugeot', 'Land Rover', 'Mercedes',
       'Audi', 'BMW', 'Mini', 'Volvo', 'Jaguar', 'Lexus', 'Infiniti',
       'Porsche', 'Cadillac', 'Lincoln', 'Maserati', 'McLaren',
       'Aston Martin', 'Bentley', 'Rolls-Royce'], dtype=object)

### origin

In [836]:
df_newcar_inf['origin'].unique()

array(['Lap rap', 'Nhap khau'], dtype=object)

### body_type, seating_capacity, engine

In [837]:
df_newcar_inf['body_type'].unique()

array(['Hatchback', 'Sedan', 'Minivan', 'SUV', 'Crossover', 'Xe ban tai',
       'Wagon', 'Van', 'Coupe', 'Convertible'], dtype=object)

In [838]:
df_newcar_inf['body_type'] = df_newcar_inf['body_type'].apply(lambda x: x.replace('Xe ban tai', 'Ban tai'))

In [839]:
df_newcar_inf['body_type'].unique()

array(['Hatchback', 'Sedan', 'Minivan', 'SUV', 'Crossover', 'Ban tai',
       'Wagon', 'Van', 'Coupe', 'Convertible'], dtype=object)

In [840]:
df_newcar_inf['seating_capacity'].unique()
df_newcar_inf['seating_capacity'] = df_newcar_inf['seating_capacity'].str.replace(' cho', '', regex=False).astype(int)

In [841]:
df_newcar_inf['engine'].unique()
df_newcar_inf['engine'] = df_newcar_inf['engine'].str.replace(' cc', '', regex=False).astype(int)

In [842]:
df_newcar_inf['engine'].unique()

array([1248,  998, 1197, 1206, 1193, 1399, 1368, 1395, 1199, 1498, 1496,
       1397, 1495, 1373, 1462, 1497, 1329, 1500, 1499, 1591, 1999, 2488,
       1353, 1598, 1599, 1596, 2198, 2393, 2457, 1796, 1898, 1364,  999,
       1000, 1798, 1799, 2396, 3198, 1998, 1996, 2402, 1988, 2359, 2999,
       1997, 2499, 2755, 1987, 1995, 2199, 2442, 2497, 2395, 2982, 1618,
       2998, 2360, 2694, 2995, 2494, 2498, 1595, 2448, 3342, 3471, 1984,
       1991, 1797, 1398, 3630, 1969, 2356, 2972, 2000, 2261, 2487, 3456,
       3778, 3498, 2979, 2996, 3696, 3457, 4608, 5663, 2894, 2993, 3649,
       3596, 4605, 4395, 2981, 5552, 3444, 3802, 4691, 6162, 3997, 5461,
       3996, 3994, 5935, 3982, 5000, 5980, 3995, 6592, 6755])

In [843]:
def round_up(x):
    return ((x + 49) // 50) * 50
df_newcar_inf['engine'] = df_newcar_inf['engine'].apply(round_up)
df_newcar_inf['engine'].unique()

array([1250, 1000, 1200, 1400, 1500, 1350, 1600, 2000, 2500, 2200, 2400,
       1800, 1900, 3200, 2450, 3000, 2800, 1650, 2700, 3350, 3500, 3650,
       2300, 3800, 3700, 4650, 5700, 2900, 3600, 4400, 5600, 3450, 3850,
       4700, 6200, 4000, 5500, 5950, 5000, 6000, 6600, 6800])

### Final

In [844]:
df_newcar_inf.columns = df_newcar_inf.columns.str.title()
df_newcar_inf.drop_duplicates()

,Name,Manufacturer,Origin,Body_Type,Seating_Capacity,Engine,Transmission,Drive,Fuel_Type,Fuel_Capacity
0,Kia Morning MT,Kia,Lap rap,Hatchback,5,1250,So san,FWD,Xang,35
1,Suzuki Celerio MT,Suzuki,Nhap khau,Hatchback,5,1000,So san,2WD,Xang,35
2,Kia Morning AT,Kia,Lap rap,Hatchback,5,1250,Tu dong,FWD,Xang,35
3,Toyota Wigo 1.2 G MT,Toyota,Nhap khau,Hatchback,5,1200,So san,FWD,Xang,33
4,Suzuki Celerio CVT,Suzuki,Nhap khau,Hatchback,5,1000,Vo cap CVT,2WD,Xang,35
...,...,...,...,...,...,...,...,...,...,...
582,Mercedes-AMG G65 6.0 V12,Mercedes,Nhap khau,SUV,5,6000,Tu dong,4WD,Xang,96
583,Aston Martin Vanquish S V12,Aston Martin,Nhap khau,Coupe,2,5950,Tu dong,RWD,Xang,77
584,Rolls-Royce Ghost V12,Rolls-Royce,Nhap khau,Sedan,5,6600,Tu dong,RWD,Xang,82
585,Rolls-Royce Cullinan V12,Rolls-Royce,Nhap khau,SUV,4,6800,Tu dong,AWD,Xang,83


In [845]:
df_newcar_inf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              587 non-null    object
 1   Manufacturer      587 non-null    object
 2   Origin            587 non-null    object
 3   Body_Type         587 non-null    object
 4   Seating_Capacity  587 non-null    int64 
 5   Engine            587 non-null    int64 
 6   Transmission      587 non-null    object
 7   Drive             587 non-null    object
 8   Fuel_Type         587 non-null    object
 9   Fuel_Capacity     587 non-null    int64 
dtypes: int64(3), object(7)
memory usage: 46.0+ KB


In [846]:
df_newcar_inf.head()

,Name,Manufacturer,Origin,Body_Type,Seating_Capacity,Engine,Transmission,Drive,Fuel_Type,Fuel_Capacity
0,Kia Morning MT,Kia,Lap rap,Hatchback,5,1250,So san,FWD,Xang,35
1,Suzuki Celerio MT,Suzuki,Nhap khau,Hatchback,5,1000,So san,2WD,Xang,35
2,Kia Morning AT,Kia,Lap rap,Hatchback,5,1250,Tu dong,FWD,Xang,35
3,Toyota Wigo 1.2 G MT,Toyota,Nhap khau,Hatchback,5,1200,So san,FWD,Xang,33
4,Suzuki Celerio CVT,Suzuki,Nhap khau,Hatchback,5,1000,Vo cap CVT,2WD,Xang,35


## Bonbanh.com

In [847]:
df_bonbanh_inf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5440 entries, 0 to 5439
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         5440 non-null   object
 1   Price        5440 non-null   object
 2   SellDate     5440 non-null   object
 3   Date         5440 non-null   object
 4   Condition    5440 non-null   object
 5   Mileage      5440 non-null   object
 6   Origin       5440 non-null   object
 7   BodyType     5440 non-null   object
 8   EngineType   5440 non-null   object
 9   Color        5440 non-null   object
 10  ColorInside  5440 non-null   object
 11  Seats        5440 non-null   object
 12  Doors        5440 non-null   object
 13  City         5440 non-null   object
dtypes: object(14)
memory usage: 595.1+ KB


### Replace None

In [848]:
df_bonbanh_inf.replace({'-': np.nan, 'None': np.nan}, inplace=True)
df_bonbanh_inf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5440 entries, 0 to 5439
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         5440 non-null   object
 1   Price        5440 non-null   object
 2   SellDate     5440 non-null   object
 3   Date         5440 non-null   object
 4   Condition    5440 non-null   object
 5   Mileage      5440 non-null   object
 6   Origin       5440 non-null   object
 7   BodyType     5440 non-null   object
 8   EngineType   5440 non-null   object
 9   Color        5436 non-null   object
 10  ColorInside  5377 non-null   object
 11  Seats        5440 non-null   object
 12  Doors        5360 non-null   object
 13  City         5440 non-null   object
dtypes: object(14)
memory usage: 595.1+ KB


### Condition   

In [849]:
df_bonbanh_inf['Condition'].unique()

array(['Xe da dung', 'Xe moi'], dtype=object)

In [850]:
df_bonbanh_inf = df_bonbanh_inf[df_bonbanh_inf['Condition'] != 'Xe moi']
df_bonbanh_inf.drop(columns=['Condition'], inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\1946601528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bonbanh_inf.drop(columns=['Condition'], inplace=True)


In [851]:
df_bonbanh_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5360 entries, 0 to 5439
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         5360 non-null   object
 1   Price        5360 non-null   object
 2   SellDate     5360 non-null   object
 3   Date         5360 non-null   object
 4   Mileage      5360 non-null   object
 5   Origin       5360 non-null   object
 6   BodyType     5360 non-null   object
 7   EngineType   5360 non-null   object
 8   Color        5359 non-null   object
 9   ColorInside  5297 non-null   object
 10  Seats        5360 non-null   object
 11  Doors        5360 non-null   object
 12  City         5360 non-null   object
dtypes: object(13)
memory usage: 586.2+ KB


### Name

#### Checking


In [852]:
filtered_cars = df_bonbanh_inf['name'].apply(lambda x: x.split(" - ")[0].strip()[-4:])
print(filtered_cars.apply(lambda x: x.isdigit()).all())
filtered_cars.unique()

False


array(['2022', '2021', '2019', '2018', '2014', '2016', '2020', '2013',
       '2023', '2010', '2024', '2017', '2012', '2015', '2009', '2006',
       '2004', '4 AT'], dtype=object)

In [853]:
filtered_cars = df_bonbanh_inf[['name']].copy()

filtered_cars['year'] = filtered_cars['name'].apply(lambda x: x.split(" - ")[0].strip()[-4:])
filtered_cars['is_valid_year'] = filtered_cars['year'].apply(lambda x: x.isdigit())

invalid_years = filtered_cars[~filtered_cars['is_valid_year']]

print(invalid_years)

                                           name  year  is_valid_year
345   Xe Honda CRV 2.4 AT - TG 2016 - 585 Trieu  4 AT          False
752   Xe Honda CRV 2.4 AT - TG 2016 - 568 Trieu  4 AT          False
2585  Xe Honda CRV 2.4 AT - TG 2017 - 605 Trieu  4 AT          False
2592  Xe Honda CRV 2.4 AT - TG 2017 - 619 Trieu  4 AT          False
3717  Xe Honda CRV 2.4 AT - TG 2017 - 635 Trieu  4 AT          False
3786  Xe Honda CRV 2.4 AT - TG 2017 - 595 Trieu  4 AT          False
5186  Xe Honda CRV 2.4 AT - TG 2017 - 625 Trieu  4 AT          False


#### Formating name

In [854]:
def extract_car_name(car_info):
    if car_info == 'Honda CRV 2.4 AT':
        return 'Xe Honda CRV 2.4'
    name_with_year = car_info.split(" - ")[0].strip()
    return name_with_year[:-5].strip()

In [855]:
df_bonbanh_inf['name'] = df_bonbanh_inf['name'].apply(lambda x: x.replace("Xe ", ""))
df_bonbanh_inf['name'] = df_bonbanh_inf['name'].apply(extract_car_name)
df_bonbanh_inf.head(10)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\2417567429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bonbanh_inf['name'] = df_bonbanh_inf['name'].apply(lambda x: x.replace("Xe ", ""))
C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\2417567429.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bonbanh_inf['name'] = df_bonbanh_inf['name'].apply(extract_car_name)


,name,Price,SellDate,Date,Mileage,Origin,BodyType,EngineType,Color,ColorInside,Seats,Doors,City
0,Hyundai Accent 1.4 AT Dac Biet,465 Trieu,4/11/2024,2022,"31,000 Km",Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi
1,Hyundai Accent 1.4 AT Dac Biet,465 Trieu,4/11/2024,2022,"31,000 Km",Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi
2,Hyundai SantaFe Cao cap 2.2L HTRAC,1 Ty 90 Trieu,4/11/2024,2021,"47,000 Km",Lap rap trong nuoc,SUV,Dau 2.2 L,Trang,Nau,7 cho,5 cua,Ha Noi
3,Hyundai SantaFe Cao cap 2.2L HTRAC,1 Ty 90 Trieu,4/11/2024,2021,"47,000 Km",Lap rap trong nuoc,SUV,Dau 2.2 L,Trang,Nau,7 cho,5 cua,TP HCM
4,MG 5 Standard 1.5 AT,410 Trieu,4/11/2024,2022,"25,000 Km",Nhap khau,Sedan,Xang 1.5 L,Den,Den,5 cho,4 cua,TP HCM
5,MG 5 Standard 1.5 AT,410 Trieu,4/11/2024,2022,"25,000 Km",Nhap khau,Sedan,Xang 1.5 L,Den,Den,5 cho,4 cua,Hai Phong
12,Hyundai Accent 1.4 AT Dac Biet,465 Trieu,4/11/2024,2022,"31,000 Km",Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi
13,Hyundai Accent 1.4 AT Dac Biet,465 Trieu,4/11/2024,2022,"31,000 Km",Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi
14,Toyota Innova 2.0E,530 Trieu,4/11/2024,2019,"60,800 Km",Lap rap trong nuoc,Crossover,Xang 2.0 L,Bac,Cam,8 cho,5 cua,Dak Lak
15,Toyota Innova 2.0E,530 Trieu,4/11/2024,2019,"60,800 Km",Lap rap trong nuoc,Crossover,Xang 2.0 L,Bac,Cam,8 cho,5 cua,Ha Noi


In [856]:
filtered_data = df_bonbanh_inf[df_bonbanh_inf['name'].str.contains('Honda CRV 2.4 AT', case=False)]

print(filtered_data)

                  name      Price   SellDate  Date    Mileage  \
850   Honda CRV 2.4 AT  495 Trieu  4/11/2024  2014  93,000 Km   
1250  Honda CRV 2.4 AT  575 Trieu  4/11/2024  2015  60,000 Km   
1838  Honda CRV 2.4 AT  620 Trieu  4/11/2024  2016  50,000 Km   
2662  Honda CRV 2.4 AT  555 Trieu  4/11/2024  2015  95,000 Km   
3128  Honda CRV 2.4 AT  548 Trieu  4/11/2024  2016  82,000 Km   
3441  Honda CRV 2.4 AT  526 Trieu  4/11/2024  2015       0 Km   
3527  Honda CRV 2.4 AT  590 Trieu  4/11/2024  2016  85,000 Km   
3787  Honda CRV 2.4 AT  462 Trieu  4/11/2024  2014  90,000 Km   

                  Origin BodyType  EngineType  Color ColorInside  Seats  \
850   Lap rap trong nuoc      SUV  Xang 2.4 L    Nau         Den  5 cho   
1250  Lap rap trong nuoc      SUV  Xang 2.4 L  Trang         Den  5 cho   
1838  Lap rap trong nuoc      SUV  Xang 2.4 L  Trang         Den  5 cho   
2662  Lap rap trong nuoc      SUV  Xang 2.4 L    Bac         Den  5 cho   
3128  Lap rap trong nuoc      SUV  Xang

### Price

In [857]:
df_bonbanh_inf['Price'].unique()

array(['465 Trieu', '1 Ty 90 Trieu', '410 Trieu', ..., '1 Ty 925 Trieu',
       '6 Ty 300 Trieu', '2 Ty 298 Trieu'], dtype=object)

In [858]:
filtered_df = df_bonbanh_inf[df_bonbanh_inf['Price'].isin(['Ha Noi', 'Dak Lak', 'TP HCM', 'Hai Phong'])]

filtered_df.head()

,name,Price,SellDate,Date,Mileage,Origin,BodyType,EngineType,Color,ColorInside,Seats,Doors,City


In [859]:
def convert_price_to_number(price_str):
    price_str = price_str.replace("Trieu", "").strip()

    parts = price_str.split(" ")

    total = 0
    if 'Ty' in parts:
        if len(parts) == 2:
            total += int(parts[0])*1000
        else:
            total += int(parts[0]) * 1000 + int(parts[2])
    else:
        total += int(parts[0])

    return total

df_bonbanh_inf['Price'] = df_bonbanh_inf['Price'].apply(convert_price_to_number)

df_bonbanh_inf.head(10)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\2514178514.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bonbanh_inf['Price'] = df_bonbanh_inf['Price'].apply(convert_price_to_number)


,name,Price,SellDate,Date,Mileage,Origin,BodyType,EngineType,Color,ColorInside,Seats,Doors,City
0,Hyundai Accent 1.4 AT Dac Biet,465,4/11/2024,2022,"31,000 Km",Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi
1,Hyundai Accent 1.4 AT Dac Biet,465,4/11/2024,2022,"31,000 Km",Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi
2,Hyundai SantaFe Cao cap 2.2L HTRAC,1090,4/11/2024,2021,"47,000 Km",Lap rap trong nuoc,SUV,Dau 2.2 L,Trang,Nau,7 cho,5 cua,Ha Noi
3,Hyundai SantaFe Cao cap 2.2L HTRAC,1090,4/11/2024,2021,"47,000 Km",Lap rap trong nuoc,SUV,Dau 2.2 L,Trang,Nau,7 cho,5 cua,TP HCM
4,MG 5 Standard 1.5 AT,410,4/11/2024,2022,"25,000 Km",Nhap khau,Sedan,Xang 1.5 L,Den,Den,5 cho,4 cua,TP HCM
5,MG 5 Standard 1.5 AT,410,4/11/2024,2022,"25,000 Km",Nhap khau,Sedan,Xang 1.5 L,Den,Den,5 cho,4 cua,Hai Phong
12,Hyundai Accent 1.4 AT Dac Biet,465,4/11/2024,2022,"31,000 Km",Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi
13,Hyundai Accent 1.4 AT Dac Biet,465,4/11/2024,2022,"31,000 Km",Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi
14,Toyota Innova 2.0E,530,4/11/2024,2019,"60,800 Km",Lap rap trong nuoc,Crossover,Xang 2.0 L,Bac,Cam,8 cho,5 cua,Dak Lak
15,Toyota Innova 2.0E,530,4/11/2024,2019,"60,800 Km",Lap rap trong nuoc,Crossover,Xang 2.0 L,Bac,Cam,8 cho,5 cua,Ha Noi


In [860]:
print(df_bonbanh_inf[df_bonbanh_inf['Price'] < 100]['Price'].unique())

[]


In [861]:
df_bonbanh_inf[df_bonbanh_inf['Price'] == 19]

,name,Price,SellDate,Date,Mileage,Origin,BodyType,EngineType,Color,ColorInside,Seats,Doors,City


In [862]:
df_bonbanh_inf[df_bonbanh_inf['Price'] == 25]

,name,Price,SellDate,Date,Mileage,Origin,BodyType,EngineType,Color,ColorInside,Seats,Doors,City


### Year, Date, SellDate

In [863]:
df_bonbanh_inf['SellDate'].unique()

array(['4/11/2024', '3/11/2024'], dtype=object)

In [864]:
df_bonbanh_inf['Date'] = df_bonbanh_inf['Date'].astype(int)
df_bonbanh_inf['Date'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\2814589890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bonbanh_inf['Date'] = df_bonbanh_inf['Date'].astype(int)


array([2022, 2021, 2019, 2018, 2014, 2016, 2020, 2013, 2023, 2010, 2024,
       2017, 2012, 2015, 2009, 2006, 2004])

In [865]:
df_bonbanh_inf = df_bonbanh_inf[df_bonbanh_inf['Date'] > 2000]

In [866]:
df_bonbanh_inf['Year'] = 2024 - df_bonbanh_inf['Date']
df_bonbanh_inf.rename(columns={'Date': 'Public Year'}, inplace=True)
df_bonbanh_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5360 entries, 0 to 5439
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         5360 non-null   object
 1   Price        5360 non-null   int64 
 2   SellDate     5360 non-null   object
 3   Public Year  5360 non-null   int64 
 4   Mileage      5360 non-null   object
 5   Origin       5360 non-null   object
 6   BodyType     5360 non-null   object
 7   EngineType   5360 non-null   object
 8   Color        5359 non-null   object
 9   ColorInside  5297 non-null   object
 10  Seats        5360 non-null   object
 11  Doors        5360 non-null   object
 12  City         5360 non-null   object
 13  Year         5360 non-null   int64 
dtypes: int64(3), object(11)
memory usage: 628.1+ KB


### Mileage

In [867]:
df_bonbanh_inf['Mileage'].unique()

array(['31,000 Km', '47,000 Km', '25,000 Km', '60,800 Km', '23,000 Km',
       '46,000 Km', '59,347 Km', '45,000 Km', '40,000 Km', '30,200 Km',
       '53,000 Km', '70,000 Km', '41,000 Km', '100,000 Km', '69,000 Km',
       '11,000 Km', '56,000 Km', '14,368 Km', '7,000 Km', '0 Km',
       '89,000 Km', '13,000 Km', '92,000 Km', '66,000 Km', '59,500 Km',
       '75,000 Km', '5,000 Km', '52,000 Km', '5 Km', '59,999 Km',
       '16,000 Km', '20,000 Km', '67,000 Km', '87,000 Km', '101,363 Km',
       '17,000 Km', '57,000 Km', '7,900 Km', '50,300 Km', '38,000 Km',
       '24,000 Km', '44,560 Km', '34,567 Km', '90,000 Km', '30,000 Km',
       '9,000 Km', '86,000 Km', '49,000 Km', '39,000 Km', '79,000 Km',
       '59,000 Km', '4,000 Km', '50,000 Km', '22,000 Km', '138,000 Km',
       '110,000 Km', '14,000 Km', '37,000 Km', '17,500 Km', '12,000 Km',
       '27,000 Km', '82,000 Km', '8,000 Km', '42,000 Km', '150,000 Km',
       '55,000 Km', '21,000 Km', '20 Km', '77,000 Km', '60,000 Km',
       

In [868]:
df_bonbanh_inf['Mileage'] = df_bonbanh_inf['Mileage'].apply(lambda x: x.replace(" Km", "").replace(",", ""))

df_bonbanh_inf['Mileage'] = df_bonbanh_inf['Mileage'].astype(int)

In [869]:
df_bonbanh_inf.head(10)

,name,Price,SellDate,Public Year,Mileage,Origin,BodyType,EngineType,Color,ColorInside,Seats,Doors,City,Year
0,Hyundai Accent 1.4 AT Dac Biet,465,4/11/2024,2022,31000,Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi,2
1,Hyundai Accent 1.4 AT Dac Biet,465,4/11/2024,2022,31000,Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi,2
2,Hyundai SantaFe Cao cap 2.2L HTRAC,1090,4/11/2024,2021,47000,Lap rap trong nuoc,SUV,Dau 2.2 L,Trang,Nau,7 cho,5 cua,Ha Noi,3
3,Hyundai SantaFe Cao cap 2.2L HTRAC,1090,4/11/2024,2021,47000,Lap rap trong nuoc,SUV,Dau 2.2 L,Trang,Nau,7 cho,5 cua,TP HCM,3
4,MG 5 Standard 1.5 AT,410,4/11/2024,2022,25000,Nhap khau,Sedan,Xang 1.5 L,Den,Den,5 cho,4 cua,TP HCM,2
5,MG 5 Standard 1.5 AT,410,4/11/2024,2022,25000,Nhap khau,Sedan,Xang 1.5 L,Den,Den,5 cho,4 cua,Hai Phong,2
12,Hyundai Accent 1.4 AT Dac Biet,465,4/11/2024,2022,31000,Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi,2
13,Hyundai Accent 1.4 AT Dac Biet,465,4/11/2024,2022,31000,Lap rap trong nuoc,Sedan,Xang 1.4 L,Cat,Kem,5 cho,4 cua,Ha Noi,2
14,Toyota Innova 2.0E,530,4/11/2024,2019,60800,Lap rap trong nuoc,Crossover,Xang 2.0 L,Bac,Cam,8 cho,5 cua,Dak Lak,5
15,Toyota Innova 2.0E,530,4/11/2024,2019,60800,Lap rap trong nuoc,Crossover,Xang 2.0 L,Bac,Cam,8 cho,5 cua,Ha Noi,5


### Origin

In [870]:
df_bonbanh_inf['Origin'].unique()
df_bonbanh_inf['Origin'] = df_bonbanh_inf['Origin'].replace("Lap rap trong nuoc", "Lap rap")

In [871]:
df_bonbanh_inf['Origin'].unique()

array(['Lap rap', 'Nhap khau'], dtype=object)

### BodyType

In [872]:
df_bonbanh_inf['BodyType'].unique()

array(['Sedan', 'SUV', 'Crossover', 'Ban tai / Pickup', 'Van/Minivan',
       'Hatchback', 'Convertible/Cabriolet', 'Coupe'], dtype=object)

In [873]:
df_bonbanh_inf['BodyType'] = df_bonbanh_inf['BodyType'].replace("Van/Minivan", "Van")
df_bonbanh_inf['BodyType'] = df_bonbanh_inf['BodyType'].replace('Convertible/Cabriolet', 'Convertible')
df_bonbanh_inf['BodyType'] = df_bonbanh_inf['BodyType'].replace('Ban tai / Pickup', 'Ban tai')

In [874]:
df_bonbanh_inf = df_bonbanh_inf[df_bonbanh_inf['BodyType'] != 'Truck']

In [875]:
df_bonbanh_inf['BodyType'].unique()

array(['Sedan', 'SUV', 'Crossover', 'Ban tai', 'Van', 'Hatchback',
       'Convertible', 'Coupe'], dtype=object)

### EngineType, Engine


In [876]:
df_bonbanh_inf['EngineType'].unique()

array(['Xang 1.4 L', 'Dau 2.2 L', 'Xang 1.5 L', 'Xang 2.0 L',
       'Xang 1.8 L', 'Dau 2.4 L', 'Xang 3.3 L', 'Xang 3.5 L',
       'Xang 1.6 L', 'Xang 2.5 L', 'Xang 1.0 L', 'Dau 2.0 L',
       'Hybrid 2.5 L', 'Dau 2.5 L', 'Hybrid 3.5 L', 'Xang 1.25 L',
       'Xang 2.7 L', 'Xang 3.0 L', 'Dien', 'Xang 1.2 L', 'Xang 2.4 L',
       'Xang 4.0 L', 'Xang 6.0 L', 'Xang 4.7 L', 'Xang 5.0 L',
       'Xang 4.6 L', 'Xang 6.7 L', 'Dien 0.3 L', 'Xang 2.3 L',
       'Dau 2.3 L', 'Xang 6.6 L', 'Hybrid 1.8 L', 'Dau 3.2 L',
       'Dien 0.2 L', 'Dau 2.8 L', 'Xang 2.9 L', 'Xang 3.4 L',
       'Hybrid 2.0 L', 'Xang 3.6 L', 'Xang 5.7 L', 'Dau 2.1 L',
       'Dien 0.1 L', 'Xang 6.8 L', 'Dien 0.6 L', 'Xang 3.9 L',
       'Dau 3.0 L', 'Xang 6.2 L', 'Dau 1.7 L', 'Xang', 'Hybrid 1.5 L',
       'Hybrid 3.0 L', 'Xang 1.3 L', 'Xang 3.8 L', 'Xang 4.4 L',
       'Dau 1.6 L', 'Xang 5.5 L', 'Xang 5.6 L'], dtype=object)

In [877]:
df_bonbanh_inf.loc[1918, 'EngineType'] = 'Dien'

In [878]:
def extract_fuel_and_engine(engine_type):
    parts = engine_type.split(' ')
    fuel_type = parts[0]
    if fuel_type == "Dien":
        engine_size = 0
    elif len(parts) > 1:
        engine_size = float(parts[1].replace('L', '')) * 1000
    else:
        engine_size = None
    return fuel_type, engine_size

df_bonbanh_inf['FuelType'], df_bonbanh_inf['Engine'] = zip(*df_bonbanh_inf['EngineType'].apply(extract_fuel_and_engine))

In [879]:
df_bonbanh_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5360 entries, 0 to 5439
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         5360 non-null   object 
 1   Price        5360 non-null   int64  
 2   SellDate     5360 non-null   object 
 3   Public Year  5360 non-null   int64  
 4   Mileage      5360 non-null   int64  
 5   Origin       5360 non-null   object 
 6   BodyType     5360 non-null   object 
 7   EngineType   5360 non-null   object 
 8   Color        5359 non-null   object 
 9   ColorInside  5297 non-null   object 
 10  Seats        5360 non-null   object 
 11  Doors        5360 non-null   object 
 12  City         5360 non-null   object 
 13  Year         5360 non-null   int64  
 14  FuelType     5360 non-null   object 
 15  Engine       5357 non-null   float64
dtypes: float64(1), int64(4), object(11)
memory usage: 840.9+ KB


In [880]:
filtered_data = df_bonbanh_inf[df_bonbanh_inf['Engine'].isnull()]
filtered_data.head()

,name,Price,SellDate,Public Year,Mileage,Origin,BodyType,EngineType,Color,ColorInside,Seats,Doors,City,Year,FuelType,Engine
1520,Audi A5 Apec,1180,4/11/2024,2017,58000,Nhap khau,Coupe,Xang,Den,Den,5 cho,0 cua,Ha Noi,7,Xang,NaN
1531,Audi Q7,830,4/11/2024,2015,100000,Nhap khau,SUV,Xang,Xam,Den,7 cho,0 cua,Ha Noi,9,Xang,NaN
3933,Subaru Outback 2.5 si,725,4/11/2024,2016,100000,Nhap khau,SUV,Xang,Den,Den,5 cho,0 cua,Ha Noi,8,Xang,NaN


In [881]:
df_bonbanh_inf.loc[691, 'Engine'] = 2000
df_bonbanh_inf.loc[2240, 'Engine'] = 1000
df_bonbanh_inf.loc[3300, 'Engine'] = 2500
df_bonbanh_inf.loc[4073, 'Engine'] = 2500
df_bonbanh_inf = df_bonbanh_inf.dropna(subset=['Engine'])
filtered_data = df_bonbanh_inf[df_bonbanh_inf['EngineType'].isnull()]
filtered_data.head()
df_bonbanh_inf['Engine'] = df_bonbanh_inf['Engine'].astype(int)

In [882]:
df_bonbanh_inf.drop(columns='EngineType', inplace=True)

### Color, ColorInside

In [883]:
df_bonbanh_inf.drop(columns=['Color'], inplace=True)
df_bonbanh_inf.drop(columns=['ColorInside'], inplace=True)

### Seats, Doors

In [884]:
df_bonbanh_inf['Seats'].unique()
df_bonbanh_inf['Seats'] = df_bonbanh_inf['Seats'].str.replace(' cho', '', regex=False).astype(int)
df_bonbanh_inf.drop(columns=['Doors'], inplace=True)

In [885]:
df_bonbanh_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5357 entries, 0 to 5439
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         5357 non-null   object
 1   Price        5357 non-null   int64 
 2   SellDate     5357 non-null   object
 3   Public Year  5357 non-null   int64 
 4   Mileage      5357 non-null   int64 
 5   Origin       5357 non-null   object
 6   BodyType     5357 non-null   object
 7   Seats        5357 non-null   int64 
 8   City         5357 non-null   object
 9   Year         5357 non-null   int64 
 10  FuelType     5357 non-null   object
 11  Engine       5357 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 544.1+ KB


### Manufacture

In [886]:
df_bonbanh_inf['Manufacturer'] = df_bonbanh_inf['name'].apply(lambda x: x.split(" ")[0])
df_bonbanh_inf['Manufacturer'].unique()

array(['Hyundai', 'MG', 'Toyota', 'Mercedes', 'Ford', 'Mitsubishi', 'Kia',
       'Lexus', 'Mini', 'Honda', 'Peugeot', 'Nissan', 'Mazda', 'Audi',
       'VinFast', 'Porsche', 'BMW', 'Chevrolet', 'Bentley', 'LandRover',
       'Rolls', 'Volkswagen', 'Volvo', 'Suzuki', 'Maserati', 'Subaru',
       'Ferrari', 'Isuzu', 'Lincoln', 'Infiniti', 'Ssangyong', 'Cadillac',
       'Lamborghini', 'McLaren', 'Jaguar', 'Skoda'], dtype=object)

In [887]:
df_bonbanh_inf['Manufacturer'] = df_bonbanh_inf['Manufacturer'].replace({'Rolls': 'Rolls-Royce'})
df_bonbanh_inf['Manufacturer'] = df_bonbanh_inf['Manufacturer'].replace({'LandRover': 'Land Rover'})
df_bonbanh_inf['Manufacturer'].unique()

array(['Hyundai', 'MG', 'Toyota', 'Mercedes', 'Ford', 'Mitsubishi', 'Kia',
       'Lexus', 'Mini', 'Honda', 'Peugeot', 'Nissan', 'Mazda', 'Audi',
       'VinFast', 'Porsche', 'BMW', 'Chevrolet', 'Bentley', 'Land Rover',
       'Rolls-Royce', 'Volkswagen', 'Volvo', 'Suzuki', 'Maserati',
       'Subaru', 'Ferrari', 'Isuzu', 'Lincoln', 'Infiniti', 'Ssangyong',
       'Cadillac', 'Lamborghini', 'McLaren', 'Jaguar', 'Skoda'],
      dtype=object)

### City

In [888]:
df_bonbanh_inf['City'].unique()

array(['Ha Noi', 'TP HCM', 'Hai Phong', 'Dak Lak', 'Ninh Binh',
       'Bac Ninh', 'Binh Duong', 'Gia Lai', 'Nghe An', 'Quang Ninh',
       'Ba Ria Vung Tau', 'Hai Duong', 'Long An'], dtype=object)

### Final

In [889]:
df_bonbanh_inf.columns = df_bonbanh_inf.columns.str.title()
df_bonbanh_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5357 entries, 0 to 5439
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          5357 non-null   object
 1   Price         5357 non-null   int64 
 2   Selldate      5357 non-null   object
 3   Public Year   5357 non-null   int64 
 4   Mileage       5357 non-null   int64 
 5   Origin        5357 non-null   object
 6   Bodytype      5357 non-null   object
 7   Seats         5357 non-null   int64 
 8   City          5357 non-null   object
 9   Year          5357 non-null   int64 
 10  Fueltype      5357 non-null   object
 11  Engine        5357 non-null   int64 
 12  Manufacturer  5357 non-null   object
dtypes: int64(6), object(7)
memory usage: 585.9+ KB


In [890]:
df_bonbanh_inf.to_csv('bonbanh_dataforml.csv')

## Oto.com

In [891]:
df_used_Car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   car_id        1174 non-null   int64 
 1   title         1174 non-null   object
 2   year          1174 non-null   object
 3   Body_Type     1174 non-null   object
 4   Origin        1174 non-null   object
 5   Mileage       1174 non-null   object
 6   City          1174 non-null   object
 7   District      1174 non-null   object
 8   Transmission  1174 non-null   object
 9   Fuel_Type     1174 non-null   object
 10  Price         1174 non-null   object
 11  sale_date     1174 non-null   object
 12  Manufacturer  1174 non-null   object
dtypes: int64(1), object(12)
memory usage: 119.4+ KB


### Replace None

In [892]:
df_used_Car.replace({'None': np.nan}, inplace=True)
df_used_Car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   car_id        1174 non-null   int64 
 1   title         1174 non-null   object
 2   year          1174 non-null   object
 3   Body_Type     992 non-null    object
 4   Origin        1174 non-null   object
 5   Mileage       1170 non-null   object
 6   City          1174 non-null   object
 7   District      1094 non-null   object
 8   Transmission  1174 non-null   object
 9   Fuel_Type     1174 non-null   object
 10  Price         1174 non-null   object
 11  sale_date     1174 non-null   object
 12  Manufacturer  1174 non-null   object
dtypes: int64(1), object(12)
memory usage: 119.4+ KB


In [893]:
df_used_Car.drop(columns='car_id', inplace=True)
df_used_Car.head()

,title,year,Body_Type,Origin,Mileage,City,District,Transmission,Fuel_Type,Price,sale_date,Manufacturer
0,2023 - Lexus LX 600,2023,SUV,Nhap khau,14000.0,Tp.HCM,Thu Duc,So tu dong,May xang,8 ti 950 trieu,22/08/2024,Lexus
1,2014 - Mercedes-Benz CLA 200 1.6 I4,2014,Sedan,Nhap khau,60000.0,Ha Noi,Nam Tu Liem,So tu dong,May xang,499 trieu,8/9/2024,Mercedes-Benz
2,2016 - Mercedes-Benz C200 2.0 AT,2016,Sedan,Trong nuoc,95000.0,Ha Noi,Nam Tu Liem,So tu dong,May xang,639 trieu,17/09/2024,Mercedes-Benz
3,2015 - Mercedes-Benz C250 AMG,2015,Sedan,Trong nuoc,100000.0,Ha Noi,Nam Tu Liem,So tu dong,May xang,666 trieu,3/9/2024,Mercedes-Benz
4,2014 - Mercedes-Benz C200 Edition C,2014,Sedan,Trong nuoc,100000.0,Ha Noi,Nam Tu Liem,So tu dong,May xang,420 trieu,2/9/2024,Mercedes-Benz


### Name

In [894]:
filtered_data = df_used_Car['title'].apply(lambda x:x.split(" - ")[0])
filtered_data.unique()

array(['2023', '2014', '2016', '2015', '2017', '2021', '2022', '2019',
       '2018', '2007', '2024', '2009', '2011', '2006', '2003', '2020',
       '2013', '2010', '2012', '2008', '2000', '1995', '1999', '2004',
       '2002', '2005'], dtype=object)

In [895]:
filtered_data = df_used_Car['title'].apply(lambda x:len(x.split(" - ")))
filtered_data.unique()

array([2])

In [896]:
df_used_Car['name'] = df_used_Car['title'].apply(lambda x:x.split(" - ")[1])
df_used_Car.drop(columns='title', inplace=True)
df_used_Car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          1174 non-null   object
 1   Body_Type     992 non-null    object
 2   Origin        1174 non-null   object
 3   Mileage       1170 non-null   object
 4   City          1174 non-null   object
 5   District      1094 non-null   object
 6   Transmission  1174 non-null   object
 7   Fuel_Type     1174 non-null   object
 8   Price         1174 non-null   object
 9   sale_date     1174 non-null   object
 10  Manufacturer  1174 non-null   object
 11  name          1174 non-null   object
dtypes: object(12)
memory usage: 110.2+ KB


### Public Year

In [897]:
df_used_Car['year'].unique()

array(['2023', '2014', '2016', '2015', '2017', '2021', '2022', '2019',
       '2018', '2007', '2024', '2009', '2011', '2006', '2003', '2020',
       '2013', '2010', '2012', '2008', '2000', '1995', '1999', '2004',
       '2002', '2005'], dtype=object)

In [898]:
df_used_Car['year'] = df_used_Car['year'].astype(float)
df_used_Car['year'] = df_used_Car['year'].astype(int)
df_used_Car['year'].unique()

array([2023, 2014, 2016, 2015, 2017, 2021, 2022, 2019, 2018, 2007, 2024,
       2009, 2011, 2006, 2003, 2020, 2013, 2010, 2012, 2008, 2000, 1995,
       1999, 2004, 2002, 2005])

In [899]:
df_used_Car.rename(columns={'year': 'Public Year'}, inplace=True)

### Origin

In [900]:
df_used_Car['Origin'].unique()

array(['Nhap khau', 'Trong nuoc'], dtype=object)

In [901]:
df_used_Car['Origin'] = df_used_Car['Origin'].replace({'Trong nuoc': 'Lap rap'})
df_used_Car['Origin'].unique()

array(['Nhap khau', 'Lap rap'], dtype=object)

### Mileage

In [902]:
df_used_Car = df_used_Car.dropna(subset=['Mileage'])

In [903]:
df_used_Car['Mileage'].unique()

array(['14000.0', '60000.0', '95000.0', '100000.0', '80000.0', '82000.0',
       '41000.0', '4000.0', '48000.0', '31000.0', '70000.0', '90000.0',
       '50000.0', '32000.0', '14600.0', '29000.0', '22900.0', '85000.0',
       '10000.0', '20000.0', '150000.0', '42000.0', '5000.0', '120000.0',
       '96000.0', '39000.0', '30000.0', '19000.0', '56000.0', '56789.0',
       '180000.0', '81234.0', '105000.0', '43000.0', '110000.0',
       '140000.0', '58000.0', '91000.0', '59999.0', '33000.0', '68000.0',
       '123456.0', '13000.0', '38000.0', '135000.0', '40000.0', '88000.0',
       '9000.0', '15000.0', '109000.0', '26000.0', '1000.0', '18000.0',
       '74000.0', '12000.0', '81000.0', '53000.0', '64000.0', '79000.0',
       '55600.0', '83000.0', '36000.0', '62000.0', '35689.0', '55000.0',
       '129000.0', '44000.0', '45000.0', '130000.0', '17000.0',
       '112000.0', '91689.0', '91268.0', '36899.0', '33689.0', '45689.0',
       '78999.0', '39899.0', '56899.0', '6846.0', '28000.0', '11

In [904]:
df_used_Car['Mileage'] = df_used_Car['Mileage'].astype(float)
df_used_Car['Mileage'] = df_used_Car['Mileage'].astype(int)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\3925398424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Mileage'] = df_used_Car['Mileage'].astype(float)
C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\3925398424.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Mileage'] = df_used_Car['Mileage'].astype(int)


### City, District

In [905]:
df_used_Car['City'].unique()

array(['Tp.HCM', 'Ha Noi', 'Hai Duong', 'Hai Phong', 'Bac Ninh',
       'Nghe An', 'Thanh Hoa', 'Quang Ninh', 'Binh Duong', 'Gia Lai',
       'Lam Dong', 'Thua Thien Hue', 'Hung Yen', 'Dak Lak', 'Nam Dinh',
       'Dong Nai', 'Long An', 'Ba Ria Vung Tau', 'Phu Tho', 'Quang Tri',
       'Dien Bien', 'Hoa Binh', 'Can Tho', 'Bac Giang', 'Thai Binh',
       'Cao Bang', 'Son La', 'Vinh Phuc', 'Binh Dinh', 'Phu Yen'],
      dtype=object)

In [906]:
df_used_Car.drop(columns='District', inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\672521784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car.drop(columns='District', inplace=True)


### Transmission

In [907]:
df_used_Car['Transmission'].unique()

array(['So tu dong', 'So san', 'So hon hop'], dtype=object)

### Fuel Type

In [908]:
df_used_Car['Fuel_Type'].unique()

array(['May xang', 'May dau', 'Hybrid', 'Dien'], dtype=object)

In [909]:
df_used_Car['Fuel_Type'] = df_used_Car['Fuel_Type'].replace({'May xang': 'Xang', 'May dau': 'Dau',})
df_used_Car['Fuel_Type'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\1625322093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Fuel_Type'] = df_used_Car['Fuel_Type'].replace({'May xang': 'Xang', 'May dau': 'Dau',})


array(['Xang', 'Dau', 'Hybrid', 'Dien'], dtype=object)

### Manufacturer

In [910]:
df_used_Car['Manufacturer'].unique()

array(['Lexus', 'Mercedes-Benz', 'Ford', 'Porsche', 'Toyota',
       'Mitsubishi', 'Nissan', 'Hyundai', 'Infiniti', 'Kia', 'Honda',
       'Chevrolet', 'VinFast', 'Bentley', 'Land Rover', 'Audi', 'BMW',
       'Daewoo', 'McLaren', 'Mazda', 'Suzuki', 'Jaguar', 'Peugeot',
       'Ferrari', 'MG', 'Volvo', 'Rolls-Royce', 'Cadillac', 'Jeep',
       'Volkswagen', 'Dodge', 'Lincoln', 'Isuzu', 'Renault', 'Mini'],
      dtype=object)

In [911]:
df_used_Car['Manufacturer'] = df_used_Car['Manufacturer'].replace({'Mercedes-Benz': 'Mercedes'})
df_used_Car['Manufacturer'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\1389325563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Manufacturer'] = df_used_Car['Manufacturer'].replace({'Mercedes-Benz': 'Mercedes'})


array(['Lexus', 'Mercedes', 'Ford', 'Porsche', 'Toyota', 'Mitsubishi',
       'Nissan', 'Hyundai', 'Infiniti', 'Kia', 'Honda', 'Chevrolet',
       'VinFast', 'Bentley', 'Land Rover', 'Audi', 'BMW', 'Daewoo',
       'McLaren', 'Mazda', 'Suzuki', 'Jaguar', 'Peugeot', 'Ferrari', 'MG',
       'Volvo', 'Rolls-Royce', 'Cadillac', 'Jeep', 'Volkswagen', 'Dodge',
       'Lincoln', 'Isuzu', 'Renault', 'Mini'], dtype=object)

### Price

In [912]:
def convert_price(price_str):
    price_str = price_str.replace(" trieu", "").strip()

    parts = price_str.split(" ")

    total = 0
    if 'ti' in parts:
        if len(parts) == 2:
            total += int(parts[0])*1000
        else:
            total += int(parts[0]) * 1000 + int(parts[2])
    else:
        total += int(parts[0])

    return total


In [913]:
df_used_Car['Price'].unique()

array(['8 ti 950 trieu', '499 trieu', '639 trieu', '666 trieu',
       '420 trieu', '1 ti 399 trieu', '445 trieu', '925 trieu',
       '1 ti 950 trieu', '995 trieu', '3 ti 999 trieu', '369 trieu',
       '450 trieu', '569 trieu', '415 trieu', '799 trieu',
       '1 ti 979 trieu', '4 ti 879 trieu', '385 trieu', '725 trieu',
       '870 trieu', '3 ti 180 trieu', '485 trieu', '880 trieu',
       '1 ti 799 trieu', '3 ti 430 trieu', '525 trieu', '690 trieu',
       '875 trieu', '660 trieu', '288 trieu', '939 trieu',
       '8 ti 340 trieu', '1 ti 125 trieu', '340 trieu', '495 trieu',
       '830 trieu', '115 trieu', '727 trieu', '3 ti 450 trieu',
       '435 trieu', '2 ti 299 trieu', '650 trieu', '388 trieu',
       '390 trieu', '555 trieu', '559 trieu', '685 trieu', '905 trieu',
       '310 trieu', '5 ti 599 trieu', '1 ti 499 trieu', '440 trieu',
       '950 trieu', '258 trieu', '989 trieu', '1 ti 920 trieu',
       '1 ti 740 trieu', '125 trieu', '2 ti 850 trieu', '1 ti 680 trieu',
       

In [914]:
df_used_Car['Price'] = df_used_Car['Price'].apply(convert_price)

df_used_Car.head(10)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\3723713032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Price'] = df_used_Car['Price'].apply(convert_price)


,Public Year,Body_Type,Origin,Mileage,City,Transmission,Fuel_Type,Price,sale_date,Manufacturer,name
0,2023,SUV,Nhap khau,14000,Tp.HCM,So tu dong,Xang,8950,22/08/2024,Lexus,Lexus LX 600
1,2014,Sedan,Nhap khau,60000,Ha Noi,So tu dong,Xang,499,8/9/2024,Mercedes,Mercedes-Benz CLA 200 1.6 I4
2,2016,Sedan,Lap rap,95000,Ha Noi,So tu dong,Xang,639,17/09/2024,Mercedes,Mercedes-Benz C200 2.0 AT
3,2015,Sedan,Lap rap,100000,Ha Noi,So tu dong,Xang,666,3/9/2024,Mercedes,Mercedes-Benz C250 AMG
4,2014,Sedan,Lap rap,100000,Ha Noi,So tu dong,Xang,420,2/9/2024,Mercedes,Mercedes-Benz C200 Edition C
5,2016,Sedan,Lap rap,80000,Ha Noi,So tu dong,Xang,1399,8/9/2024,Mercedes,Mercedes-Benz S400 3.0 V6
6,2017,SUV,Lap rap,82000,Tp.HCM,So tu dong,Xang,445,19/09/2024,Ford,Ford Focus 1.5L EcoBoost Sport
7,2021,SUV,Lap rap,41000,Tp.HCM,So tu dong,Xang,925,19/09/2024,Ford,Ford Everest Titanium 2.0 AT 4x2
8,2022,SUV,Nhap khau,4000,Tp.HCM,So tu dong,Xang,1950,19/09/2024,Ford,Ford Explorer Limited
9,2022,Ban Tai,Nhap khau,48000,Tp.HCM,So tu dong,Dau,995,15/09/2024,Ford,Ford Ranger Raptor


In [915]:
print(df_used_Car[df_used_Car['Price'] < 100]['Price'].unique())

[70 79 75 68 98 95 96 90 59]


### Year

In [916]:
df_used_Car['sale_date'].unique()

array(['22/08/2024', '8/9/2024', '17/09/2024', '3/9/2024', '2/9/2024',
       '19/09/2024', '15/09/2024', '6/9/2024', '31/08/2024', '29/08/2024',
       '5/9/2024', '23/08/2024', '30/08/2024', '9/9/2024', '20/09/2024',
       '7/9/2024', '4/9/2024', '12/9/2024', '14/09/2024', '13/09/2024',
       '28/08/2024', '16/09/2024', '27/08/2024', '11/9/2024',
       '18/09/2024', '10/9/2024', '23/09/2024', '25/08/2024',
       '26/08/2024', '25/09/2024', '1/9/2024', '24/08/2024', '24/09/2024',
       '22/09/2024'], dtype=object)

In [917]:
df_used_Car['Year'] = 2024 - df_used_Car['Public Year']
df_used_Car['Year'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\2461185671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Year'] = 2024 - df_used_Car['Public Year']


array([ 1, 10,  8,  9,  7,  3,  2,  5,  6, 17,  0, 15, 13, 18, 21,  4, 11,
       14, 12, 16, 24, 29, 25, 20, 22, 19])

### Body_type

In [918]:
df_used_Car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1170 entries, 0 to 1173
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Public Year   1170 non-null   int64 
 1   Body_Type     988 non-null    object
 2   Origin        1170 non-null   object
 3   Mileage       1170 non-null   int64 
 4   City          1170 non-null   object
 5   Transmission  1170 non-null   object
 6   Fuel_Type     1170 non-null   object
 7   Price         1170 non-null   int64 
 8   sale_date     1170 non-null   object
 9   Manufacturer  1170 non-null   object
 10  name          1170 non-null   object
 11  Year          1170 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 118.8+ KB


In [919]:
def find_body_type2(row):
    match = process.extractOne(row['name'], df_newcar_inf['Name'], score_cutoff=90)
    if match and match[1] > 90:
        print(f"Tên xe cũ: '{row['name']}' - Tên xe mới: '{match[0]}' - Điểm tương đồng: {match[1]}")
        index = df_newcar_inf[df_newcar_inf['Name'] == match[0]].index
        return df_newcar_inf.loc[index, 'Body_Type'].values[0]
    return None

df_used_Car['Body_Type'] = df_used_Car.apply(lambda row: find_body_type2(row) if pd.isna(row['Body_Type']) else row['Body_Type'], axis=1)

Tên xe cũ: 'Toyota Corolla Cross 1.8 G' - Tên xe mới: 'Toyota Corolla Cross 1.8 G' - Điểm tương đồng: 100
Tên xe cũ: 'Land Rover Range Rover' - Tên xe mới: 'Land Rover Range Rover Velar S' - Điểm tương đồng: 95
Tên xe cũ: 'Porsche Macan S' - Tên xe mới: 'Porsche Macan S' - Điểm tương đồng: 100
Tên xe cũ: 'Porsche Macan S' - Tên xe mới: 'Porsche Macan S' - Điểm tương đồng: 100
Tên xe cũ: 'Porsche Macan' - Tên xe mới: 'Porsche Macan 2.0' - Điểm tương đồng: 95
Tên xe cũ: 'Land Rover Range Rover Evoque 2.0 R-Dynamic SE' - Tên xe mới: 'Land Rover Range Rover Evoque 2.0L I4 Turbocharged R-Dynamic SE' - Điểm tương đồng: 92
Tên xe cũ: 'Toyota Corolla Cross 1.8 V' - Tên xe mới: 'Toyota Corolla Cross 1.8 HV' - Điểm tương đồng: 98
Tên xe cũ: 'Hyundai Santa Fe 2.2 Dau Dac biet' - Tên xe mới: 'Hyundai SantaFe 2.2 Dau dac biet' - Điểm tương đồng: 98
Tên xe cũ: 'Land Rover Range Rover AutoBiography LWB 3.0L' - Tên xe mới: 'Land Rover Range Rover Autobiography LWB' - Điểm tương đồng: 95
Tên xe cũ: 'Ch

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\1741481242.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Body_Type'] = df_used_Car.apply(lambda row: find_body_type2(row) if pd.isna(row['Body_Type']) else row['Body_Type'], axis=1)


In [920]:
def find_body_type3(row):
    match = process.extractOne(row['name'], df_bonbanh_inf['Name'], score_cutoff=90)
    if match and match[1] > 90:
        print(f"Tên xe cũ: '{row['name']}' - Tên xe mới: '{match[0]}' - Điểm tương đồng: {match[1]}")
        index = df_bonbanh_inf[df_bonbanh_inf['Name'] == match[0]].index
        return df_bonbanh_inf.loc[index, 'Bodytype'].values[0]
    return None

df_used_Car['Body_Type'] = df_used_Car.apply(lambda row: find_body_type3(row) if pd.isna(row['Body_Type']) else row['Body_Type'], axis=1)

Tên xe cũ: 'Lexus RX 350' - Tên xe mới: 'Lexus RX 350' - Điểm tương đồng: 100
Tên xe cũ: 'Lexus LX 570' - Tên xe mới: 'Lexus LX 570' - Điểm tương đồng: 100
Tên xe cũ: 'Toyota Vios' - Tên xe mới: 'Toyota Vios 1.5G' - Điểm tương đồng: 95
Tên xe cũ: 'Lexus RX 200t' - Tên xe mới: 'Lexus RX 200t' - Điểm tương đồng: 100
Tên xe cũ: 'Mercedes-Benz Maybach S450' - Tên xe mới: 'Mercedes Benz Maybach S400' - Điểm tương đồng: 96
Tên xe cũ: 'Mazda 6 2.0 Premium' - Tên xe mới: 'Mazda 6 2.0L Premium' - Điểm tương đồng: 97
Tên xe cũ: 'VinFast VF9 Plus' - Tên xe mới: 'VinFast VF9 Plus' - Điểm tương đồng: 100
Tên xe cũ: 'Mercedes-Benz GLS 450' - Tên xe mới: 'Mercedes Benz GLS 450 4Matic' - Điểm tương đồng: 95
Tên xe cũ: 'Mercedes-Benz C250' - Tên xe mới: 'Mercedes Benz C class C250' - Điểm tương đồng: 95
Tên xe cũ: 'Lexus LM 350' - Tên xe mới: 'Lexus LM 350' - Điểm tương đồng: 100
Tên xe cũ: 'Toyota Camry 2.0E' - Tên xe mới: 'Toyota Camry 2.0E' - Điểm tương đồng: 100
Tên xe cũ: 'Peugeot 2008 GT Line' - 

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\2792897167.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Body_Type'] = df_used_Car.apply(lambda row: find_body_type3(row) if pd.isna(row['Body_Type']) else row['Body_Type'], axis=1)


In [921]:
def find_body_type(row, df):
    filtered_df = df[df['name'] != row['name']]

    match = process.extractOne(row['name'], filtered_df['name'], score_cutoff=87)
    
    if match and match[1] > 87:
        similar_car_name = match[0]
        print(f"Tên xe cũ: '{row['name']}' - Tên xe tương tự: '{similar_car_name}' - Điểm tương đồng: {match[1]}")

        body_type = df.loc[df['name'] == similar_car_name, 'Body_Type'].values[0]
        return body_type

    return None

df_used_Car['Body_Type'] = df_used_Car.apply(
    lambda row: find_body_type(row, df_used_Car) if pd.isna(row['Body_Type']) else row['Body_Type'], 
    axis=1
)

Tên xe cũ: 'Honda CR-V' - Tên xe tương tự: 'Honda CR-V 2.0' - Điểm tương đồng: 95
Tên xe cũ: 'Honda City 1.5 TOP CVT' - Tên xe tương tự: 'Honda City 1.5 CVT' - Điểm tương đồng: 95
Tên xe cũ: 'Honda CR-V' - Tên xe tương tự: 'Honda CR-V 2.0' - Điểm tương đồng: 95
Tên xe cũ: 'Mazda 3 1.5L Sedan' - Tên xe tương tự: 'Mazda 3 Sedan 1.5L Luxury' - Điểm tương đồng: 95
Tên xe cũ: 'Peugeot 2008' - Tên xe tương tự: 'Peugeot 5008' - Điểm tương đồng: 92
Tên xe cũ: 'Ford Transit' - Tên xe tương tự: 'Ford Transit SVP Ban trung' - Điểm tương đồng: 90
Tên xe cũ: 'VinFast LUX A2.0' - Tên xe tương tự: 'VinFast LUX A2.0 Cao cap' - Điểm tương đồng: 90
Tên xe cũ: 'Mazda 3' - Tên xe tương tự: 'Mazda 3 Sedan 1.5L Luxury' - Điểm tương đồng: 90
Tên xe cũ: 'Volkswagen Scirocco 1.4' - Tên xe tương tự: 'Volkswagen Scirocco' - Điểm tương đồng: 95
Tên xe cũ: 'Honda CR-V' - Tên xe tương tự: 'Honda CR-V 2.0' - Điểm tương đồng: 95
Tên xe cũ: 'Ford EcoSport' - Tên xe tương tự: 'Ford EcoSport Titanium 1.0L AT' - Điểm tươ

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\2399812481.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_used_Car['Body_Type'] = df_used_Car.apply(


In [922]:
df_used_Car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1170 entries, 0 to 1173
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Public Year   1170 non-null   int64 
 1   Body_Type     1158 non-null   object
 2   Origin        1170 non-null   object
 3   Mileage       1170 non-null   int64 
 4   City          1170 non-null   object
 5   Transmission  1170 non-null   object
 6   Fuel_Type     1170 non-null   object
 7   Price         1170 non-null   int64 
 8   sale_date     1170 non-null   object
 9   Manufacturer  1170 non-null   object
 10  name          1170 non-null   object
 11  Year          1170 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 118.8+ KB


In [923]:
df_used_Car['name'][df_used_Car['Body_Type'].isna()]

42                  Toyota Zace
51                  Toyota Zace
66            Daewoo Lacetti SE
498                 Toyota Zace
505     Volkswagen Scirocco 1.4
521                  Ford Laser
544                Toyota Hiace
573            MG 5 1.5L Luxury
589                Renault Wind
784                 Nissan Juke
1143        Volkswagen Scirocco
1153                Peugeot 408
Name: name, dtype: object

In [924]:
df_used_Car.loc[521, 'Body_Type'] = 'Sedan'
df_used_Car.loc[573, 'Body_Type'] = 'Crossover'
df_used_Car.loc[589, 'Body_Type'] = 'Convertible'
df_used_Car.loc[784, 'Body_Type'] = 'SUV'
df_used_Car.loc[1153, 'Body_Type'] = 'Crossover'

In [925]:
df_used_Car['name'][df_used_Car['Body_Type'].isna()]

42                  Toyota Zace
51                  Toyota Zace
66            Daewoo Lacetti SE
498                 Toyota Zace
505     Volkswagen Scirocco 1.4
544                Toyota Hiace
1143        Volkswagen Scirocco
Name: name, dtype: object

In [926]:
df_used_Car['Body_Type'].unique()

array(['SUV', 'Sedan', 'Ban Tai', 'Hatchback', None, 'MPV', 'Van/Minivan',
       'Crossover', 'Sport Car', 'Coupe', 'Convertible', 'Minibus',
       'Ban tai', 'Van', 'Xe tai'], dtype=object)

### Final

In [927]:
df_used_Car.columns = df_used_Car.columns.str.title()

In [928]:
df_used_Car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1170 entries, 0 to 1173
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Public Year   1170 non-null   int64 
 1   Body_Type     1163 non-null   object
 2   Origin        1170 non-null   object
 3   Mileage       1170 non-null   int64 
 4   City          1170 non-null   object
 5   Transmission  1170 non-null   object
 6   Fuel_Type     1170 non-null   object
 7   Price         1170 non-null   int64 
 8   Sale_Date     1170 non-null   object
 9   Manufacturer  1170 non-null   object
 10  Name          1170 non-null   object
 11  Year          1170 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 151.1+ KB


# Merge data

In [929]:
df_used_Car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1170 entries, 0 to 1173
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Public Year   1170 non-null   int64 
 1   Body_Type     1163 non-null   object
 2   Origin        1170 non-null   object
 3   Mileage       1170 non-null   int64 
 4   City          1170 non-null   object
 5   Transmission  1170 non-null   object
 6   Fuel_Type     1170 non-null   object
 7   Price         1170 non-null   int64 
 8   Sale_Date     1170 non-null   object
 9   Manufacturer  1170 non-null   object
 10  Name          1170 non-null   object
 11  Year          1170 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 151.1+ KB


In [930]:
df_bonbanh_inf = df_bonbanh_inf.rename(columns={'Bodytype': 'Body_Type', 'Selldate':'Sale_Date', 'Fueltype': 'Fuel_Type'})

In [931]:
df_bonbanh_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5357 entries, 0 to 5439
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          5357 non-null   object
 1   Price         5357 non-null   int64 
 2   Sale_Date     5357 non-null   object
 3   Public Year   5357 non-null   int64 
 4   Mileage       5357 non-null   int64 
 5   Origin        5357 non-null   object
 6   Body_Type     5357 non-null   object
 7   Seats         5357 non-null   int64 
 8   City          5357 non-null   object
 9   Year          5357 non-null   int64 
 10  Fuel_Type     5357 non-null   object
 11  Engine        5357 non-null   int64 
 12  Manufacturer  5357 non-null   object
dtypes: int64(6), object(7)
memory usage: 715.0+ KB


In [932]:
df_bonbanh_inf.describe()

,Price,Public Year,Mileage,Seats,Year,Engine
count,5357.000000,5357.000000,5.357000e+03,5357.000000,5357.000000,5357.000000
mean,1346.485346,2019.655031,5.018477e+04,5.572335,4.344969,2151.810715
std,1575.819671,2.625224,1.228567e+05,1.147229,2.625224,851.792955
min,155.000000,2004.000000,0.000000e+00,2.000000,0.000000,0.000000
25%,545.000000,2018.000000,2.300000e+04,5.000000,2.000000,1600.000000
50%,785.000000,2020.000000,4.600000e+04,5.000000,4.000000,2000.000000
75%,1399.000000,2022.000000,6.900000e+04,7.000000,6.000000,2400.000000
max,26999.000000,2024.000000,8.000000e+06,47.000000,20.000000,6800.000000


Can merge: Name, Public Year, Mileage, Body_Type, Origin, City, Fuel_Type, Sale_Date, Manufacturer, Year, Price \\\
Error: Seats, Engine, Transmission

In [933]:
# merge_keys = ['Name', 'Public Year', 'Mileage', 'Body_Type', 'Origin', 'City', 'Fuel_Type', 'Sale_Date', 'Manufacturer', 'Year', 'Price']

# final_data = pd.merge(df_bonbanh_inf, df_used_Car, on=merge_keys, how='outer')
# final_data.info()

In [934]:
# def find_engine(row, df):
#     filtered_df = df[df['Name'] != row['Name']]

#     match = process.extractOne(row['Name'], filtered_df['Name'], score_cutoff=90)
    
#     if match and match[1] > 87:
#         similar_car_name = match[0]
#         print(f"Tên xe cũ: '{row['Name']}' - Tên xe tương tự: '{similar_car_name}' - Điểm tương đồng: {match[1]}")

#         engine_values = df.loc[df['Name'] == similar_car_name, ['Engine', 'Public Year']]
        
#         if not engine_values.empty:
#             engine_value = engine_values.iloc[0]['Engine']
#             similar_year = engine_values.iloc[0]['Public Year']

#             if similar_year == row['Public Year']:
#                 return engine_value
        
#         print(f"Không tìm thấy giá trị Engine cho tên xe '{similar_car_name}' hoặc năm không phù hợp.")

#     return None

# final_data['Engine'] = final_data.apply(
#     lambda row: find_engine(row, final_data) if pd.isna(row['Engine']) else row['Engine'], 
#     axis=1
# )

In [935]:
final_data = df_bonbanh_inf

In [936]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5357 entries, 0 to 5439
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          5357 non-null   object
 1   Price         5357 non-null   int64 
 2   Sale_Date     5357 non-null   object
 3   Public Year   5357 non-null   int64 
 4   Mileage       5357 non-null   int64 
 5   Origin        5357 non-null   object
 6   Body_Type     5357 non-null   object
 7   Seats         5357 non-null   int64 
 8   City          5357 non-null   object
 9   Year          5357 non-null   int64 
 10  Fuel_Type     5357 non-null   object
 11  Engine        5357 non-null   int64 
 12  Manufacturer  5357 non-null   object
dtypes: int64(6), object(7)
memory usage: 715.0+ KB


In [937]:
Q1 = final_data['Price'].quantile(0.25)
Q3 = final_data['Price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

final_data = final_data[(final_data['Price'] >= lower_bound) & (final_data['Price'] <= upper_bound)]
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4713 entries, 0 to 5439
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          4713 non-null   object
 1   Price         4713 non-null   int64 
 2   Sale_Date     4713 non-null   object
 3   Public Year   4713 non-null   int64 
 4   Mileage       4713 non-null   int64 
 5   Origin        4713 non-null   object
 6   Body_Type     4713 non-null   object
 7   Seats         4713 non-null   int64 
 8   City          4713 non-null   object
 9   Year          4713 non-null   int64 
 10  Fuel_Type     4713 non-null   object
 11  Engine        4713 non-null   int64 
 12  Manufacturer  4713 non-null   object
dtypes: int64(6), object(7)
memory usage: 515.5+ KB


In [938]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4713 entries, 0 to 5439
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          4713 non-null   object
 1   Price         4713 non-null   int64 
 2   Sale_Date     4713 non-null   object
 3   Public Year   4713 non-null   int64 
 4   Mileage       4713 non-null   int64 
 5   Origin        4713 non-null   object
 6   Body_Type     4713 non-null   object
 7   Seats         4713 non-null   int64 
 8   City          4713 non-null   object
 9   Year          4713 non-null   int64 
 10  Fuel_Type     4713 non-null   object
 11  Engine        4713 non-null   int64 
 12  Manufacturer  4713 non-null   object
dtypes: int64(6), object(7)
memory usage: 515.5+ KB


### Formating


In [939]:
for column in ['Origin', 'Body_Type', 'City', 'Fuel_Type', 'Manufacturer']:
    unique_values = final_data[column].unique()
    print(f"Giá trị duy nhất trong '{column}': {unique_values}")

Giá trị duy nhất trong 'Origin': ['Lap rap' 'Nhap khau']
Giá trị duy nhất trong 'Body_Type': ['Sedan' 'SUV' 'Crossover' 'Ban tai' 'Van' 'Hatchback' 'Convertible'
 'Coupe']
Giá trị duy nhất trong 'City': ['Ha Noi' 'TP HCM' 'Hai Phong' 'Dak Lak' 'Ninh Binh' 'Bac Ninh'
 'Binh Duong' 'Gia Lai' 'Nghe An' 'Quang Ninh' 'Ba Ria Vung Tau'
 'Hai Duong' 'Long An']
Giá trị duy nhất trong 'Fuel_Type': ['Xang' 'Dau' 'Hybrid' 'Dien']
Giá trị duy nhất trong 'Manufacturer': ['Hyundai' 'MG' 'Toyota' 'Mercedes' 'Ford' 'Mitsubishi' 'Kia' 'Mini'
 'Honda' 'Peugeot' 'Nissan' 'Lexus' 'Mazda' 'Audi' 'VinFast' 'BMW'
 'Chevrolet' 'Bentley' 'Land Rover' 'Volkswagen' 'Porsche' 'Suzuki'
 'Volvo' 'Subaru' 'Isuzu' 'Infiniti' 'Ssangyong' 'Jaguar' 'Maserati'
 'Lincoln' 'Skoda']


In [940]:
final_data['City'] = final_data['City'].replace({
    'Tp.HCM': 'TP HCM'
})

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\3328532522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['City'] = final_data['City'].replace({


### Binning

In [941]:
final_data.describe()

,Price,Public Year,Mileage,Seats,Year,Engine
count,4713.000000,4713.000000,4.713000e+03,4713.000000,4713.000000,4713.000000
mean,889.503925,2019.657119,5.121911e+04,5.564821,4.342881,1961.500106
std,529.661509,2.638036,1.304379e+05,0.929890,2.638036,578.090479
min,155.000000,2004.000000,0.000000e+00,2.000000,0.000000,0.000000
25%,525.000000,2018.000000,2.400000e+04,5.000000,2.000000,1500.000000
50%,720.000000,2020.000000,4.700000e+04,5.000000,4.000000,2000.000000
75%,1090.000000,2022.000000,7.000000e+04,7.000000,6.000000,2200.000000
max,2680.000000,2024.000000,8.000000e+06,8.000000,20.000000,6200.000000


In [942]:
nins = [0, 300, 800, 2000, float('inf')]

final_data['Price-binned'] = pd.qcut(final_data['Price'], q=4, labels=['Low', 'Medium', 'High', 'Luxury'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_26584\1394480854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['Price-binned'] = pd.qcut(final_data['Price'], q=4, labels=['Low', 'Medium', 'High', 'Luxury'])


In [943]:
final_data['Price-binned'].unique()

['Low', 'High', 'Medium', 'Luxury']
Categories (4, object): ['Low' < 'Medium' < 'High' < 'Luxury']

### Data Normalization (Min - max scaling)

### Encode(One-hot-encoding)

In [944]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4713 entries, 0 to 5439
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Name          4713 non-null   object  
 1   Price         4713 non-null   int64   
 2   Sale_Date     4713 non-null   object  
 3   Public Year   4713 non-null   int64   
 4   Mileage       4713 non-null   int64   
 5   Origin        4713 non-null   object  
 6   Body_Type     4713 non-null   object  
 7   Seats         4713 non-null   int64   
 8   City          4713 non-null   object  
 9   Year          4713 non-null   int64   
 10  Fuel_Type     4713 non-null   object  
 11  Engine        4713 non-null   int64   
 12  Manufacturer  4713 non-null   object  
 13  Price-binned  4713 non-null   category
dtypes: category(1), int64(6), object(7)
memory usage: 520.3+ KB


In [945]:
final_data_encoded = pd.get_dummies(final_data, columns=['Origin', 'Body_Type', 'City', 'Fuel_Type', 'Manufacturer', 'Price-binned'])

In [946]:
final_data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4713 entries, 0 to 5439
Data columns (total 70 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Name                     4713 non-null   object
 1   Price                    4713 non-null   int64 
 2   Sale_Date                4713 non-null   object
 3   Public Year              4713 non-null   int64 
 4   Mileage                  4713 non-null   int64 
 5   Seats                    4713 non-null   int64 
 6   Year                     4713 non-null   int64 
 7   Engine                   4713 non-null   int64 
 8   Origin_Lap rap           4713 non-null   bool  
 9   Origin_Nhap khau         4713 non-null   bool  
 10  Body_Type_Ban tai        4713 non-null   bool  
 11  Body_Type_Convertible    4713 non-null   bool  
 12  Body_Type_Coupe          4713 non-null   bool  
 13  Body_Type_Crossover      4713 non-null   bool  
 14  Body_Type_Hatchback      4713 non-null   bool

Map boolean to 0 1

In [947]:
bool_columns = final_data_encoded.select_dtypes(include=['bool']).columns
final_data_encoded[bool_columns] = final_data_encoded[bool_columns].astype(int)
final_data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4713 entries, 0 to 5439
Data columns (total 70 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Name                     4713 non-null   object
 1   Price                    4713 non-null   int64 
 2   Sale_Date                4713 non-null   object
 3   Public Year              4713 non-null   int64 
 4   Mileage                  4713 non-null   int64 
 5   Seats                    4713 non-null   int64 
 6   Year                     4713 non-null   int64 
 7   Engine                   4713 non-null   int64 
 8   Origin_Lap rap           4713 non-null   int64 
 9   Origin_Nhap khau         4713 non-null   int64 
 10  Body_Type_Ban tai        4713 non-null   int64 
 11  Body_Type_Convertible    4713 non-null   int64 
 12  Body_Type_Coupe          4713 non-null   int64 
 13  Body_Type_Crossover      4713 non-null   int64 
 14  Body_Type_Hatchback      4713 non-null   int6

# Store data

## Create table

In [948]:
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE final_data (
        Name NVARCHAR(100),
        Price FLOAT,
        Sale_Date NVARCHAR(50),
        Public_Year INT,
        Mileage FLOAT,
        Origin NVARCHAR(100),
        Body_Type NVARCHAR(100),
        Seats INT,
        City NVARCHAR(100),
        Year INT,
        Fuel_Type NVARCHAR(50),
        Engine INT,
        Manufacturer NVARCHAR(100),
        Price_binned NVARCHAR(50)
    )
''')

conn.commit()

## Insert

In [949]:
cursor.execute('DELETE FROM final_data')

for index, row in final_data.iterrows():
    cursor.execute(''' 
        INSERT INTO final_data (Name, Price, Sale_Date, Public_Year, Mileage, Origin, Body_Type, Seats, City, Year, Fuel_Type, Engine, Manufacturer, Price_binned) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', 
    row['Name'], row['Price'], row['Sale_Date'], row['Public Year'], row['Mileage'], 
    row['Origin'], row['Body_Type'], row['Seats'], row['City'], row['Year'], 
    row['Fuel_Type'], row['Engine'], row['Manufacturer'], row['Price-binned'])

conn.commit()
cursor.close()
conn.close()
